Equação de posição:


In [ ]:
import scipy 
from scipy.spatial import distance

class Road:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.length = distance.euclidean(self.start, self.end)
        self.angle_sin = (self.end[1]-self.start[1]) / self.length
        self.angle_cos = (self.end[0]-self.start[0]) / self.length
        self.adjacent_roads = []
        self.vehicles = []

    def add_adjacent_road(self, road):
        self.add_adjacent_road.append(road)

    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)

    def remove_vehicle(self, vehicle):
        self.vehicles.remove(vehicle)

In [ ]:
class Functions():
    
    def position_equation(self, initial_position:float, velocity:float, time:float) -> float:  #posição
        return initial_position + velocity*time

    def zero_array(self, length: int) -> list:
        
        result_array = []
        iterator = 0 
        
        while iterator < length:
            result_array.append(0)
            iterator += 1
        
        return result_array    
    
    def amax_like(array_to_find_max, axis ):
        return None
    
    def vstack_like():
        return None
    
    def root_function(self, number_to_root, )
    def sqrt_like(self, number_to_sqrt):
        return None

    

functions = Functions()

In [ ]:
class Vehicle():
    def __init__(self, guided_by, desired_speed, initial_speed, time_between_cars, maximum_acceleration
    , maximum_slowdown, acceleration_exponent, minimum_interval, simulation):
        self.guided_by = guided_by
        self.desired_speed = desired_speed #v0
        self.initial_speed = initial_speed #v_inicial
        self.time_between_cars = time_between_cars #T
        self.maximum_acceleration = maximum_acceleration #a
        self.maximum_slowdown = maximum_slowdown #b
        self.acceleration_exponent = acceleration_exponent #delta
        self.minimum_interval = minimum_interval #s0 in meters
        self.track_length = simulation.track_length
        self.track_length = simulation.track_length #fim_da_pista in meters
        self.time_steps = simulation.time_step #passos_tempo -> 
        self.total_time = simulation.total_time #tempo_total
        self.correction_factor = simulation.correction_factor #c
        self.initial_time = simulation.initial_time #t_inicial
        self.time_step = simulation.time_step
        #self.number_of_cars = number_of_cars
    
    def s_asterisco(self, delta_v,v):
        zeros =  functions.zero_array(length = len(delta_v))
        return self.minimum_interval + np.amax(np.vstack((zeros, v*self.time_between_cars + v*delta_v/(2*functions.sqrt_like(self.maximum_acceleration * self.maximum_slowdown)))), axis = 0)
        #return s0 + np.amax(np.vstack((zeros,v*T + v*delta_v/(2*np.sqrt(a*b)))),axis=0) #Eq. 11.15
    
    def a_IDM(self, v,s,delta_v): #IDM
        return self.maximum_acceleration*(1-(v/self.desired_speed)**(self.acceleration_exponent) - (self.s_asterisco(v, delta_v)/self.s)**2) #Eq. 11.14
    
    def a_free_function(self, v):#page 196
        
        comp = (v <= self.desired_speed) # 1 or 0
        
        return (comp*(self.maximum_acceleration*(1-(v/self.desired_speed)**self.acceleration_exponent))
        +(1-comp)*(-self.maximum_slowdown*(1-(self.desired_speed/v)**
        (self.maximum_acceleration*self.acceleration_exponent/self.maximum_slowdown)))) #Eq. 11.22

    def a_IIDM(self, v,s,delta_v): #Improved IDM

        z = self.s_asterisco(v = v, delta_v = delta_v)/s
        v_comparation = (v <= self.desired_speed) # 1 or 0
        z_comparation = (z >= 1)
        a_free = self.a_free_function(v)
        dv_dt = np.zeros(v.shape)

        #Eq. 11.23 e 11.24
        dv_dt += v_comparation * z_comparation * (self.maximum_acceleration*(1-z**2))
        # Com v <= v0 e z < 1
        dv_dt += v_comparation * (1-z_comparation) * (a_free*(1-z**(2*self.maximum_acceleration/a_free)))
        # Com v > v0 e z >= 1
        dv_dt += (1-v_comparation) * z_comparation * (a_free+self.maximum_acceleration*(1-z**2))
        # Com v > v0 e z < 1
        dv_dt += (1-v_comparation) * z_comparation * (a_free)

        return dv_dt

    def a_CAH(self, s, v, vl, v_dash_l): #constant-acceleration heuristic Eq. 11.25
        ã_l = np.minimum(v_dash_l, self.maximum_acceleration)
        comp = ((vl*(v-vl))<=(-2*s*ã_l))
        return ((np.square(v)*ã_l/(np.square(vl) - 2*s*ã_l) * comp) + (ã_l - (np.square(v-vl)*((v-vl)>0)/(2*s)))*(1-comp))

    def a_ACC(self, s, v, vl, a_iidm): #ACC Eq. 11.26 

        v_dash_l = np.roll(a_iidm,1)
        a_cah = self.a_CAH(s, v, vl, v_dash_l)
        comp = (a_iidm >= a_cah) #1 or 0
        return ((a_iidm * comp) + ((1-self.correction_factor)*a_iidm +  self.correction_factor
        *(a_cah + self.maximum_slowdown*np.tanh((a_iidm-a_cah)/self.maximum_slowdown)))*(1-comp))

    def x_v_dash(self, x_v, t, simulation):
        x_v = x_v.reshape(2,-2) # derivada da posição e velocidade
        v = x_v[1,:] # obter velocidades
        x_vec = x_v[0,:] # obter posições
        # Pista circular, carro 0 segue o carro n-1, delta_v = v[i] - v[i-1]
        vl = np.roll(v,1)
        delta_v = v - vl # diferença entre as velocidades
        s = np.roll(x_vec,1) - x_vec
        s[0] += self.track_length
        if self.simulation.model_number == 1:
            dvdt = self.a_IDM(v,s,delta_v)
        else:
            dvdt = self.a_IIDM(v,s,delta_v)
        if self.simulation.model_number == 2:
            dvdt = self.a_ACC(s, v, vl, dvdt)
        x_v = np.concatenate((v,dvdt))
        return x_v

    def runge_kutta_4(self, x_v_vec_k, t_k, h, simulation): #Runge-Kutta 4th Order 
        k1=self.x_v_dash(x_v_vec_k,t_k)
        k2=self.x_v_dash(x_v_vec_k+0.5*h*k1,t_k+0.5*h)
        k3=self.x_v_dash(x_v_vec_k+0.5*h*k2,t_k+0.5*h)
        k4=self.x_v_dash(x_v_vec_k+h*k3,t_k+h)
        x_v_vec_k_next = x_v_vec_k + h/6.0 * (k1 + 2*k2 + 2*k3 + k4)
        simulation.x_v_dash.append(k1) 
        return x_v_vec_k_next

#class IDM(Vehicle):
#class HDM(Vehicle):        
    


In [ ]:

class Simulation():
    def __init__(self, model_number, number_of_cars, track_length, time_step, total_time, correction_factor, initial_time):
        self.model_number = model_number
        self.number_of_cars = number_of_cars
        self.track_length = track_length
        self.track_length = track_length #fim_da_pista in meters
        self.time_steps = time_step #passos_tempo -> 
        self.total_time = total_time #tempo_total
        self.correction_factor = correction_factor #c
        self.initial_time = initial_time #t_inicial
        self.time_step = (self.total_time - self.initial_time)/self.time_steps
        self.vehicle = vehicle = Vehicle(guided_by = "Human", desired_speed= 50.0, initial_speed= 5.0, time_between_cars= 1.0,
        maximum_acceleration= 2.0, maximum_slowdown= 3.0, acceleration_exponent= 4.0, minimum_interval= 2.0, simulation = self )
        self.x_v_dash = [2*np.zeros(self.number_of_cars)] #existe um argumento e uma função com o msm nome, isso deixou extremamente confuso, consertar
        self.roads = []

    def create_road(self, start, end):
        road = Road(start, end)
        self.road.append(road)
    

    def run_simulation(self, params):
        #need to create a vehicle
        v = np.ones(self.number_of_cars) * self.vehicle.initial_speed
        x_vec = np.linspace(0,self.track_length-self.track_length/6,self.number_of_cars)
        x_vec = x_vec[::-1] # inverter as posições 
        x_v_vec = np.concatenate(([x_vec], [v]), axis=0).reshape(1,-1)[0] # criar um vetor de velocidade
        ts = np.linspace(0,self.total_time,self.time_step)
        y_s = []
        y_s.append(x_v_vec)
        for i in range(1,len(ts)):
            y_s.append(self.vehicle.runge_kutta_4(y_s[-1], self.x_v_dash, ts[i], ts[i]-ts[i-1]))
        y_s = np.array(y_s)
        return y_s

    def main_function(self):
        

        for model_in_use in range(2): #0 == IDM / 1 == IIDM / 2 == ACC
            self.model_number = model_in_use
            v = np.ones(self.number_of_cars) * self.vehicle.initial_speed
            x_vec = np.linspace(0,self.track_length-self.track_length/6,self.number_of_cars)
            x_vec = x_vec[::-1]
            x_v_vec = np.concatenate(([x_vec], [v]), axis=0).reshape(1,-1)[0]
            ts = np.linspace(0,self.total_time,self.time_steps)
            y_s=[]
            y_s.append(x_v_vec)
            for i in range(1,len(ts)):
                y_s.append(self.vehicle.runge_kutta_4(y_s[-1], self.x_v_dash, ts[i], ts[i]-ts[i-1]))
            y_s = np.array(y_s)
            # Plot posição e velocidade de cada carro
            fig, axes = plt.subplots(1,2, figsize=(16,8))
            for car in range(self.number_of_cars):
                axes[0].plot(ts, y_s[:,car])
            # Plot velocidade em função do tempo
            for car in range(self.number_of_cars):
                axes[1].plot(ts, y_s[:,car+self.number_of_cars])
            axes[0].set_xlabel('Tempo')
            axes[0].set_ylabel('Deslocamento') 
            axes[1].set_xlabel('Tempo')
            axes[1].set_ylabel('Velocidade')
            plt.show()
            # Plot carros rodando em circulos
            r = (self.track_length/(2*np.pi))
            fx = lambda x_vec: r*np.sin((x_vec/self.track_length)*2*np.pi)
            fy = lambda x_vec: r*np.cos((x_vec/self.track_length)*2*np.pi)
            fig, ax = plt.subplots()
            t = np.arange(0, self.track_length, dtype=np.float32)
            t_pos = np.linspace(0, 2*np.pi, self.number_of_cars)
            line, = ax.plot(fx(t), fy(t))
            cars = ax.scatter(fx(x_vec), fy(x_vec), c='g')
        def animate(i):
            x_vec = y_s[i,:self.number_of_cars]
            x_vec = np.remainder(x_vec, self.track_length)
            new_pos = np.concatenate(([fx(x_vec)], [fy(x_vec)]), axis=0)
            cars.set_offsets(new_pos.T)
            return cars,
        def init():
            return cars,
        ani = animation.FuncAnimation(fig, animate, range(self.time_steps), init_func=init, interval=1, blit=False)
        ax.set_ylabel('$x$')
        ax.set_ylabel('$y$')
        plt.show()

HDM:

In [ ]:
from collections import deque 

class HDM(Vehicle):
    def __init__(self, guided_by, desired_speed, initial_speed, time_between_cars, maximum_acceleration, maximum_slowdown, acceleration_exponent, minimum_interval, simulation):
        super().__init__(guided_by, desired_speed, initial_speed, time_between_cars, maximum_acceleration, maximum_slowdown, acceleration_exponent, minimum_interval, simulation)
        self.Tr = Tr 
        self.Vs = Vs 
        self.sigma_r = sigma_r
        self.sigma_a = sigma_a
        self.tau_til = tau_til
        self.tau_a_til = tau_a_til
        self.n_a = 3
        self.w_s = self.wiener_process(tau_til=self.tau_til)
        self.w_l = self.wiener_process(tau_til=self.tau_til)
        self.w_a = self.wiener_process(tau_til=self.tau_a_til)
        self.j = int(self.Tr/self.simulation.time_step)
        self.r = self.Tr/self.simulation.time_step - self.j
        self.past_v_l_est_s = deque(maxlen=self.j+2)
        self.past_delta_v_est_s = deque(maxlen=self.j+2)
        self.past_dvdt = deque(maxlen=self.j+2)
        self.v = np.ones(self.simulation.number_of_cars)*self.initial_speed
        self.x_vec =  np.linspace(0,self.simulation.track_length-self.simulation.track_length/5,self.simulation.number_of_cars)
        self.x_vec = self.x_vec[::-1]
        self.ts = np.linspace(self.simulation.initial_time,self.simulation.total_time,self.simulation.time_steps)
        self.x_v_vec = np.concatenate(([self.x_vec], [self.v]), axis=0).reshape(1,-1)[0]
        self.y_s = []
        self.y_s.append(self.x_v_vec)

    def interp(self, u, i):
        return self.r*u[i-self.j-1] + (1-self.r)*u[i-self.j]

    def wiener_process(self, tau_til):
        w0 = np.random.randn(self.simulation.number_of_cars)
        w = np.zeros((self.simulation.number_of_cars, self.simulation.time_steps))
        w[:,0] = w0
        for time_step in range(1, int(self.time_steps)):
            w[:, self.time_step] = np.exp(-self.time_step/tau_til)*w[:, self.time_step-1] + np.sqrt(2*self.time_step/tau_til)*np.random.randn(self.simulation.number_of_cars)
        return w
    
    def a_IDM_free(self, v): #Eq. 12.21
        return self.maximum_acceleration*(1-(v/self.initial_speed)**self.acceleration_exponent)

    def a_IDM_int(self,v,s,delta_v): #Eq. 12.21
        return -self.maximum_acceleration*(self.s_asterisco(v=v, delta_v=delta_v)/s)**2

    def x_v_dash(self, x_v, t):
        x_v = x_v.reshape(2,-2)
        v = x_v[1,:]
        x_vec = x_v[0,:]
        s = np.roll(x_vec, 1) - x_vec
        s[0] += self.simulation.track_length
        index = math.floor(t/self.simulation.time_step) if math.floor(t/self.simulation.time_step) < self.simulation.time_steps else self.simulation.time_steps - 1
        s_est = s*np.exp(self.Vs * self.w_s[:,index])
        v_l_est = np.roll(v,1) - s*self.sigma_r*self.w_l[:, index]
        delta_v_est = v - v_l_est
        dvdt = self.a_IDM(v=v, s=s_est, delta_v=delta_v_est)+self.sigma_a*self.w_a[:, index]
        x_v = np.concatenate((v, dvdt))
        return x_v

   

    def x_v_dash2(self, x_v, t, past):
        x_v = x_v.reshape(2,-2)
        v = x_v[1,:]
        x_vec = x_v[0,:]
        s = np.roll(x_vec, 1) - x_vec
        s[0] += self.simulation.track_length
        index = math.floor(t/self.simulation.time_step) if math.floor(t/self.simulation.time_step) < self.simulation.time_steps else self.simulation.time_steps - 1
        s_est = s*np.exp(self.Vs * self.w_s[:,index])
        v_l_est = np.roll(v,1) - s*self.sigma_r*self.w_l[:, index]
        past['past_v_l_est_s'].append(v_l_est)
        delta_v_est = v - v_l_est
        past['past_delta_v_est_s'].append(delta_v_est)
        past_v_l_est_s = np.array(past['past_v_l_est_s'])
        past_v_est_s = np.roll(past_v_l_est_s,-1)
        past_dvdt = np.array(past['past_dvdt'])
        past_delta_v_est_s = np.array(past['past_delta_v_est_s'])

        if past_v_est_s.shape[0] > int(self.Tr/self.simulation.time_step)+1:
            v_l_prog = self.interp(past_v_l_est_s,0)
            v_prog = self.interp(past_v_est_s,0) + self.Tr * self.interp(past_dvdt,0)
            s_prog = s_est - self.Tr * self.interp(past_delta_v_est_s,0)
            c_idm = np.sum([1./j**2 for j in range(1, self.n_a+1)])**(-1)
            free_term = self.a_IDM_free(v)
            dvdt = np.zeros(v.shape)
            for alpha in range(self.simulation.number_of_cars):  #Eq 12.20
                free_term_alpha = free_term[alpha]
                int_term = 0.0
                v_alpha_prog = np.array([v_prog[alpha]])
                for beta in range(alpha-self.n_a,alpha):
                    sum_idxs = np.arange(beta+1,alpha+1)
                    s_alpha_beta_prog = np.array([np.sum(s_prog[sum_idxs])])
                    v_beta_prog = np.array([v_l_prog[beta]])
                    int_term += self.a_IDM_int(v_alpha_prog,s_alpha_beta_prog,v_alpha_prog-v_beta_prog)
                    dvdt[alpha] = free_term_alpha + c_idm*int_term
        else:
            dvdt = self.a_IDM(v,s_est,delta_v_est) + self.sigma_a*self.w_a[:, index]
        
        x_v = np.concatenate((v,dvdt))
        past['past_dvdt'].append(dvdt)
        return x_v

    def runge_kutta_4(self,x_v_vec_k, x_v_dash, t_k, h,past):
            k1=self.x_v_dash2(x_v_vec_k,t_k,past)
            k2=self.x_v_dash2(x_v_vec_k+0.5*h*k1,t_k+0.5*h,past)
            k3=self.x_v_dash2(x_v_vec_k+0.5*h*k2,t_k+0.5*h,past)
            k4=self.x_v_dash2(x_v_vec_k+h*k3,t_k+h,past)
            x_v_vec_k_next = x_v_vec_k + h/6.0 * (k1 + 2*k2 + 2*k3 + k4)
            return x_v_vec_k_next

